# RAG进阶操作

In [ ]:
!pip install langchain_chroma -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import os
os.environ["DEEPSEEK_API_BASE"] = 'https://api.deepseek.com/v1'
os.environ["DEEPSEEK_API_KEY"] = 'sk-60e9910f323c46c3aeb645951b2d45ec'

In [ ]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

### 读取文件夹内容

In [ ]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_text_splitters import SpacyTextSplitter

### 指定文件夹路径 

In [ ]:
directory_path = r'./RAG_data2'

### 创建一个DirectoryLoader对象

In [ ]:
text_loader_kwargs = {"autodetect_encoding": True}

loader = DirectoryLoader(directory_path, glob="**/*.txt", loader_cls=TextLoader, show_progress=True, loader_kwargs=text_loader_kwargs)

### 加载文件夹中的所有文件

In [ ]:
documents = loader.load()

In [ ]:
print(len(documents))

In [ ]:
from langchain_text_splitters import __all__
print(__all__)

### 使用文本分割器将文档分割成更小的部分

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100,separators=["\n", "!", "?", "。", "！", "？"],keep_separator=False )
splitted_docs = text_splitter.split_documents(documents)

In [ ]:
print(len(splitted_docs))

In [ ]:
print(splitted_docs)

In [ ]:
os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-9OiJkk755GLEpfNU41E93135B6634035B3F1A98e0dC1Cb60'

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    splitted_docs,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
all_docs = vectorstore.get()
print(all_docs.keys())  

In [ ]:
# 获取 Chroma 存储的所有文档
all_data = vectorstore.get(include=['documents', 'metadatas', 'embeddings'])

# 取前3条文本内容
for i in range(3):
    print(f"Text: {all_data['documents'][i]}")
    print(f"Metadata: {all_data['metadatas'][i]}")
    print(f"ids: {all_data['ids'][i]}")
    print(f"embeddings: {all_data['embeddings'][i]}")

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("介绍一下互联网中如何保护儿童信息安全？")

print(response.content)

### 1. 父文档检索器 （Parent Document Retriever）

In [ ]:
from langchain_classic.retrievers import ParentDocumentRetriever

In [ ]:
from langchain_core.stores import InMemoryStore

In [ ]:
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)

child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)

store = InMemoryStore()

In [ ]:
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

In [28]:
retriever.add_documents(documents)

In [29]:
len(list(store.yield_keys()))

41

In [30]:
sub_docs = vectorstore.similarity_search("数据标注")

In [31]:
print(sub_docs[0].page_content)

（二）涉及知识产权的，不得侵害他人依法享有的知识产权；

（三）涉及个人信息的，应当取得个人同意或者符合法律、行政法规规定的其他情形；

（四）采取有效措施提高训练数据质量，增强训练数据的真实性、准确性、客观性、多样性；

（五）《中华人民共和国网络安全法》、《中华人民共和国数据安全法》、《中华人民共和国个人信息保护法》等法律、行政法规的其他有关规定和有关主管部门的相关监管要求。

第八条 在生成式人工智能技术研发过程中进行数据标注的，提供者应当制定符合本办法要求的清晰、具体、可操作的标注规则；开展数据标注质量评估，抽样核验标注内容的准确性；对标注人员进行必要培训，提升尊法守法意识，监督指导标注人员规范开展标注工作。

第三章 服务规范

第九条 提供者应当依法承担网络信息内容生产者责任，履行网络信息安全义务。涉及个人信息的，依法承担个人信息处理者责任，履行个人信息保护义务。


In [32]:
retrieved_docs = retriever.invoke("数据标注")

In [33]:
len(retrieved_docs[1].page_content)

1959

In [ ]:
print(retrieved_docs[0].page_content)

### 2. 多查询检索

In [34]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(documents)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

In [35]:
from langchain_classic.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "如何保护用户的信息安全?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [36]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain_classic.retrievers.multi_query").setLevel(logging.INFO)

In [37]:
unique_docs = retriever_from_llm.invoke(question)
len(unique_docs)

INFO:langchain_classic.retrievers.multi_query:Generated queries: ['1. 如何确保用户的个人信息不被泄露或滥用？', '2. 在数字平台中，如何加强用户数据的安全保护？', '3. 用户隐私安全的最佳防护措施有哪些？']


12

In [38]:
print(unique_docs[0].page_content)

第四十二条  网络运营者不得泄露、篡改、毁损其收集的个人信息；未经被收集者同意，不得向他人提供个人信息。但是，经过处理无法识别特定个人且不能复原的除外。

网络运营者应当采取技术措施和其他必要措施，确保其收集的个人信息安全，防止信息泄露、毁损、丢失。在发生或者可能发生个人信息泄露、毁损、丢失的情况时，应当立即采取补救措施，按照规定及时告知用户并向有关主管部门报告。

第四十三条  个人发现网络运营者违反法律、行政法规的规定或者双方的约定收集、使用其个人信息的，有权要求网络运营者删除其个人信息；发现网络运营者收集、存储的其个人信息有错误的，有权要求网络运营者予以更正。网络运营者应当采取措施予以删除或者更正。

第四十四条  任何个人和组织不得窃取或者以其他非法方式获取个人信息，不得非法出售或者非法向他人提供个人信息。

第四十五条  依法负有网络安全监督管理职责的部门及其工作人员，必须对在履行职责中知悉的个人信息、隐私和商业秘密严格保密，不得泄露、出售或者非法向他人提供。


In [ ]:
rag_chain = {"context": retriever_from_llm, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("如何保护用户的信息安全?")

print(response.content)

### 3. Ensemble Retriever

### 安装 rank-bm25包

In [ ]:
!pip install rank-bm25 -i https://pypi.tuna.tsinghua.edu.cn/simple

In [39]:
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [40]:
bm25_retriever = BM25Retriever.from_documents(
    documents[0:6]
)

In [ ]:
pip install faiss-cpu -i https://pypi.tuna.tsinghua.edu.cn/simple

In [41]:
embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_documents(
    documents[6:],embedding
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

In [42]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [43]:
docs = ensemble_retriever.invoke("儿童")
docs

[Document(metadata={'source': 'RAG_data2\\互联网信息服务深度合成管理规定.txt'}, page_content='第一章 总则\n\n第一条 为了加强互联网信息服务深度合成管理，弘扬社会主义核心价值观，维护国家安全和社会公共利益，保护公民、法人和其他组织的合法权益，根据《中华人民共和国网络安全法》、《中华人民共和国数据安全法》、《中华人民共和国个人信息保护法》、《互联网信息服务管理办法》等法律、行政法规，制定本规定。\n\n第二条 在中华人民共和国境内应用深度合成技术提供互联网信息服务（以下简称深度合成服务），适用本规定。法律、行政法规另有规定的，依照其规定。\n\n第三条 国家网信部门负责统筹协调全国深度合成服务的治理和相关监督管理工作。国务院电信主管部门、公安部门依据各自职责负责深度合成服务的监督管理工作。\n\n地方网信部门负责统筹协调本行政区域内的深度合成服务的治理和相关监督管理工作。地方电信主管部门、公安部门依据各自职责负责本行政区域内的深度合成服务的监督管理工作。\n\n第四条 提供深度合成服务，应当遵守法律法规，尊重社会公德和伦理道德，坚持正确政治方向、舆论导向、价值取向，促进深度合成服务向上向善。\n\n第五条 鼓励相关行业组织加强行业自律，建立健全行业标准、行业准则和自律管理制度，督促指导深度合成服务提供者和技术支持者制定完善业务规范、依法开展业务和接受社会监督。\n\n第二章 一般规定\n\n第六条 任何组织和个人不得利用深度合成服务制作、复制、发布、传播法律、行政法规禁止的信息，不得利用深度合成服务从事危害国家安全和利益、损害国家形象、侵害社会公共利益、扰乱经济和社会秩序、侵犯他人合法权益等法律、行政法规禁止的活动。\n\n深度合成服务提供者和使用者不得利用深度合成服务制作、复制、发布、传播虚假新闻信息。转载基于深度合成服务制作发布的新闻信息的，应当依法转载互联网新闻信息稿源单位发布的新闻信息。\n\n第七条 深度合成服务提供者应当落实信息安全主体责任，建立健全用户注册、算法机制机理审核、科技伦理审查、信息发布审核、数据安全、个人信息保护、反电信网络诈骗、应急处置等管理制度，具有安全可控的技术保障措施。\n\n第八条 深度合成服务提供者应当制定和公开管理规则、平台公约，完善服务协议，依法依约履行管理责

In [44]:
len(docs)

8